<a href="https://colab.research.google.com/github/Bk073/Word2Vec/blob/main/Word2Vec_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import io
import re
import string
import tqdm

import tensorflow as tf
from tensorflow.keras import layers

import random
from collections import Counter
import random
import numpy as np
import itertools
import functools

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

### Process data

In [4]:
def read_file(file_path):
  import codecs
  with codecs.open(file_path, 'r', encoding='latin1',errors='ignore') as fdata:
    data = fdata.read()
  return data
        
  # with open(file_path, 'r') as f:
  #   for line in f:
  #     line = bytes(line, 'utf-8').decode('utf-8', 'ignore')
  #     content += line
  # return content

In [8]:
# Download 20-news group data
# file_path = tf.keras.utils.get_file(fname='wiki_dump', origin='http://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2', extract=True, cache_subdir='/content/drive/MyDrive/word2vec')

In [9]:
# import tarfile

# t_file = tarfile.open('/content/drive/MyDrive/word2vec/20_news_group.tar.gz')
# t_file.extractall('/content/drive/MyDrive/word2vec/20_news_group')
# t_file.close()

In [10]:
def data_dump(root_path):
  train_data_list = ''
  for i in os.listdir(root_path):
    f = os.path.join(root_path, i)
    for j in os.listdir(f):
      # train_data_list.append(os.path.join(f, j))
      train_data_list += read_file(os.path.join(f, j))
      # except:
        # import pdb
        # pdb.set_trace()
  
  return train_data_list

In [ ]:
# train_dir = '/content/drive/MyDrive/word2vec/20_news_group/20news-bydate-train'
# word_list = data_dump(train_dir)

In [ ]:
# with open('/content/drive/MyDrive/word2vec/words.txt', 'w') as f:
#   f.write(word_list)

In [6]:
raw_file_path =  '/content/drive/MyDrive/word2vec/words.txt'
# test
# raw_file_path = '/content/drive/MyDrive/word2vec/text.txt'
with open(raw_file_path, 'r') as f:
  raw_file = f.read()

In [7]:
def preprocess_text(sentence, lang):
  # lower = sentence.lower()
  # email_removed = tf.strings.regex_replace(
  #     lower, "\S*@\S*\s?", ""
  # )
  # number_removed = tf.strings.regex_replace(
  #     email_removed, "[0-9]", ' '
  # )
  # punctuation_removed =  tf.strings.regex_replace(
  #     number_removed, '[%s]' % re.escape(string.punctuation), ' '
  # )
  # multiple_space_removed =tf.strings.reduce_join(tf.strings.split(punctuation_removed), separator=" ")
  # return multiple_space_removed
  if lang == 'eng':
    punctuation_ = set(string.punctuation)
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    sentence = sentence.lower()
    sentence = re.sub(r'\S*@\S*\s?', ' ', sentence)
    sentence = re.sub(r'[0-9]', ' ', sentence)
    # sentence = ''.join(ch for ch in sentence if ch not in punctuation_)
    # sentence = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+\ *', ' ', sentence)
    # sentence = re.sub(r"[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+\ *", " ", sentence)
    sentence = sentence.translate(translator)
    sentence = re.sub(r'\n', ' ', sentence)
    sentence = re.sub(r'\t+', ' ', sentence)
    sentence = re.sub(' +', ' ', sentence)
    sentence = sentence.strip()
    return sentence



In [8]:
preprocessed_data = preprocess_text(raw_file, 'eng')
print(preprocessed_data[:100])

from mathew subject alt atheism faq atheist resources summary books addresses music anything related


In [9]:
def subsampling(train_data, threshold=1e-5):
  total_words = train_data.split(' ')
  print(f"total words: {len(total_words)}")
  print(f"unique words: {len(set(total_words))}")

  c = Counter (total_words)
  total_counts = len(total_words)
  freq = {word: (count / total_counts) for word, count in c.items()}
  probability = {word: (1-np.sqrt(threshold/freq[word])) for word in c}

  train_words = [word for word in total_words if 0.2 < (1 - probability[word])]
  print(f"subsampled vocab length: {len(train_words)}")

  return train_words, total_words


In [10]:
subsampled_words, total_words = subsampling(preprocessed_data)

total words: 3300978
unique words: 77710
subsampled vocab length: 1245470


In [11]:
subsampled_words[:20]

['mathew',
 'alt',
 'atheism',
 'faq',
 'atheist',
 'resources',
 'summary',
 'books',
 'addresses',
 'music',
 'related',
 'atheism',
 'faq',
 'atheism',
 'books',
 'music',
 'fiction',
 'addresses',
 'contacts',
 'expires']

In [12]:
train_data = tf.data.TextLineDataset(raw_file_path)

In [13]:
for text in train_data.take(2):
  print(text.numpy())

b'From: mathew <mathew@mantis.co.uk>'
b'Subject: Alt.Atheism FAQ: Atheist Resources'


In [14]:
def preprocess(text):
  lower = tf.strings.lower(text)
  # remove emails
  email_removed = tf.strings.regex_replace(
      lower, "\S*@\S*\s?", ""
  )
  # remove numbers
  number_removed = tf.strings.regex_replace(
      email_removed, "[0-9]", ' '
  )
  tab_removed = tf.strings.regex_replace(number_removed, "\t+", " ")
  # remove punctuations
  punctuation_removed =  tf.strings.regex_replace(
      tab_removed, '[%s]' % re.escape(string.punctuation), ' '
  )
  # remove multiple blank spaces
  # multiple_space_removed =tf.strings.reduce_join(tf.strings.split(punctuation_removed), separator=" ")
  multiple_space_removed = tf.strings.regex_replace(punctuation_removed, ' +', ' ')
  return multiple_space_removed


In [15]:
vocab_size = len(set(subsampled_words))
sequence_length = 10

vectorize_layers = layers.TextVectorization(
    standardize=preprocess,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [16]:
vectorize_layers.adapt(train_data.batch(1024))
# vectorize_layers.adapt(train_words)

In [17]:
print(vectorize_layers.vocabulary_size())
print(vectorize_layers.get_vocabulary()[:20])

77292
['', '[UNK]', 'the', 'to', 'of', 'a', 'ax', 'and', 'i', 'in', 'is', 'that', 'it', 'for', 'you', 's', 'from', 't', 'on', 'this']


In [18]:
inverse_vocab = vectorize_layers.get_vocabulary()
inverse_vocab[-20:]

['affectionately',
 'affectionate',
 'affection',
 'affaire',
 'affadavit',
 'afew',
 'afeiqy',
 'afeiqx',
 'afeiqvf',
 'afeial',
 'afeiafei',
 'afdzqvg',
 'afdu',
 'afdod',
 'afarensis',
 'afal',
 'afair',
 'afa',
 'aether',
 'aetana']

In [19]:
len(inverse_vocab)

77292

In [20]:
text_vector_ds = train_data.batch(1024).prefetch(AUTOTUNE).map(vectorize_layers).unbatch()

In [21]:
# Sub sampling
for i in text_vector_ds.take(2):
  print(f"Vector: {i.numpy()} => word: {[inverse_vocab[x] for x in i]}")

Vector: [  16 3558    0    0    0    0    0    0    0    0] => word: ['from', 'mathew', '', '', '', '', '', '', '', '']
Vector: [  31 1032 1070  503 1597 1480    0    0    0    0] => word: ['subject', 'alt', 'atheism', 'faq', 'atheist', 'resources', '', '', '', '']


In [22]:
print(inverse_vocab[3558])

if inverse_vocab[3558] in subsampled_words:
  print("True")
else:
  print("False")

mathew
True


In [23]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

511617


In [24]:
if inverse_vocab[-1] in subsampled_words:
  print("true")

true


In [37]:
for x in inverse_vocab:
  print(x)
  break

In [24]:
subsampled_words = list(set(subsampled_words))
len(list(set(subsampled_words)))

77292

In [25]:
## list of sub-sample but with integer number from inverse_vocab
sub_sample_vocab_id = [inverse_vocab.index(word) for word in inverse_vocab if (word in subsampled_words)]

In [31]:
import pickle
with open('/content/drive/MyDrive/word2vec/sub_sample_vocab_id.txt', 'wb') as f:
  pickle.dump(sub_sample_vocab_id, f)

In [ ]:
with open('/content/drive/MyDrive/word2vec/sub_sample_vocab_id.txt', 'rb') as f:
  sub_sample_vocab_id = pickle.load(f)

### Creating vocab

In [21]:
# vocab = Counter(set(total_words))
id_word = {id:word for id, word in enumerate(inverse_vocab)}
word_id = {word:id for id, word in id_word.items()}

### Skip-gram with sub-sampling

In [28]:
def changed_skip_gram(context_window, index_vocab):
  context_vector = []
  # negative_vector = []
  sub_sample_vocab_id_local = sub_sample_vocab_id
  try:
    padding_index = index_vocab.tolist().index(0)
    for index, target in enumerate(index_vocab):
      # context = index_vocab[ index - context_window between index + context_window ] except index
      # TODO
      # check if the target's probability < threshold
      if (target in sub_sample_vocab_id_local):
        if (index<padding_index):
          left_index = index-context_window if (index-context_window) >= 0 else 0
          right_index = index+context_window+1 if (index+context_window) < padding_index else padding_index
          context_vector.extend([[target, index_vocab[c]] for c in range(left_index, right_index) if (c != index and index_vocab[c] in sub_sample_vocab_id_local)])
        # elif index > padding_index-context_window-1 and index < padding_index:
          # left_index = index-context_window if (index-context_window) >= 0 else 0
          # right_index = index+context_window+1 if (index+context_window+1) <= padding_index else len(padding_index)
          # context_vector.extend([[index_vocab[index], index_vocab[c]] for c in range(left_index, right_index) if c != index])
      
      else:
        continue
  except:
    for index, target in enumerate(index_vocab):
    # context = index_vocab[ index - context_window between index + context_window ] except index
    # TODO
    # Check probability i.e sub-sampling need to be done
      if (target in sub_sample_vocab_id_local):
          left_index = index-context_window if (index-context_window) >= 0 else 0
          right_index = index+context_window+1 if (index+context_window) < len(index_vocab) else len(index_vocab)
          context_vector.extend([[target, index_vocab[c]] for c in range(left_index, right_index) if (c != index and index_vocab[c] in sub_sample_vocab_id_local)])
      else:
        continue

  context_vector.sort()
  context_vector = list(k for k, _ in itertools.groupby(context_vector))
  return context_vector

  #   negative_sample = negative_sampling(target, context_vector, negative_sample_number, vocab, vocab_size)
  #   negative_vector.extend(negative_sample)
  # return context_vector, negative_vector

In [59]:
def skip_gram(index_vocab, context_window):
  context_vector = []
  inverse_vocab_local = inverse_vocab
  subsampled_words_local = subsampled_words
  # negative_vector = []
  try:
    padding_index = index_vocab.tolist().index(0)
    for index, target in enumerate(index_vocab):
      # context = index_vocab[ index - context_window between index + context_window ] except index
      # TODO
      # check if the target's probability < threshold
      if (inverse_vocab_local[target] in subsampled_words_local):
        if (index<padding_index):
          left_index = index-context_window if (index-context_window) >= 0 else 0
          right_index = index+context_window+1 if (index+context_window) < padding_index else padding_index
          context_vector.extend([[index_vocab[index], index_vocab[c]] for c in range(left_index, right_index) if (c != index and inverse_vocab_local[index_vocab[c]] in subsampled_words_local)])
        # elif index > padding_index-context_window-1 and index < padding_index:
          # left_index = index-context_window if (index-context_window) >= 0 else 0
          # right_index = index+context_window+1 if (index+context_window+1) <= padding_index else len(padding_index)
          # context_vector.extend([[index_vocab[index], index_vocab[c]] for c in range(left_index, right_index) if c != index])
      
      else:
        continue
  except:
    for index, target in enumerate(index_vocab):
    # context = index_vocab[ index - context_window between index + context_window ] except index
    # TODO
    # Check probability i.e sub-sampling need to be done
      if (inverse_vocab_local[target] in subsampled_words_local):
          left_index = index-context_window if (index-context_window) >= 0 else 0
          right_index = index+context_window+1 if (index+context_window) < len(index_vocab) else len(index_vocab)
          context_vector.extend([[index_vocab[index], index_vocab[c]] for c in range(left_index, right_index) if (c != index and inverse_vocab_local[index_vocab[c]] in subsampled_words_local)])
      else:
        continue

  context_vector.sort()
  context_vector = list(k for k, _ in itertools.groupby(context_vector))
  return context_vector

  #   negative_sample = negative_sampling(target, context_vector, negative_sample_number, vocab, vocab_size)
  #   negative_vector.extend(negative_sample)
  # return context_vector, negative_vector

In [60]:
example_sentence = [31, 1032, 1070, 503, 1597, 1480]
window_size=2
# positive_skip_grams_2 = skip_gram(example_sentence, window_size)
positive_skip_grams_2 = changed_skip_gram(window_size, example_sentence)
print(positive_skip_grams_2)
print(len(positive_skip_grams_2))

[[503, 1032], [503, 1070], [503, 1480], [503, 1597], [1032, 503], [1032, 1070], [1070, 503], [1070, 1032], [1070, 1597], [1480, 503], [1480, 1597], [1597, 503], [1597, 1070], [1597, 1480]]
14


In [131]:
if inverse_vocab[31] in subsampled_words:
  print("True")

In [95]:
for i in example_sentence:
  if inverse_vocab[31] in subsampled_words:
    print("True")

### Negative sampling

In [32]:
## negative sampling
def negative_sampling(target, context, negative_sample_number, vocab_size):
  # vocab_size = len(vocab)
  # TODO
# this implementation continues in loop when context_window is greater
# suppose we have vocab size of 4 [1,2,3,4] and context_window = 3 and negative sample = 2
# then we can not get a negative sample of size 2 and the while loop will continue forever
# this need to be handle
  negative_sample = []
  # negative_sample_number = 2
  # vocab_size = 7
  # vocab = index_vocab
  # or get the context for particular index and sample negative values from vocab except the context
  while len(negative_sample) < negative_sample_number:
    import pdb
    # pdb.set_trace()
    sample = random.randint(0, vocab_size-1)
    # if inverse_vocab[sample] in subsampled_words:
    if (sample != context) and (sample not in negative_sample) and sample != target:
      negative_sample.append(sample)
  return tf.convert_to_tensor(negative_sample, dtype='int64')

### Generate training data

In [61]:
def skip_gram_with_negative_sampling(sequences,context_window, negative_sample_number, vocab_size):
  targets, contexts, labels = [],[],[]
  for sequence in tqdm.tqdm(sequences):
  # for positive_skip_grams in tqdm.tqdm(map(functools.partial(changed_skip_gram, window_size), sequences)):
    import pdb
    # TODO
    # map function all at once and loop, rather making function call each time
    positive_skip_grams = skip_gram(sequence, context_window)
    # positive_skip_grams = changed_skip_gram(window_size, sequence)
    # pdb.set_trace()
    if positive_skip_grams:
      for target_word, context_word in positive_skip_grams:
        context_class = tf.expand_dims(tf.constant([context_word], dtype='int64'), 1)
        negative_sampling_candidate = negative_sampling(target_word, context_word, negative_sample_number, vocab_size)

        negative_sampling_candidate = tf.expand_dims(negative_sampling_candidate, 1)
        context = tf.concat([context_class, negative_sampling_candidate], 0)
        label = tf.constant([1] + [0]*negative_sample_number, dtype="int64")

        targets.append(target_word)
        contexts.append(context)
        labels.append(label)
    else:
      continue
  
  return targets, contexts, labels


In [36]:
# Mapping sampled training words to int

training_word_int = [word_id[word] for word in train_words]
training_word_int[:10]

NameError: ignored

In [ ]:
targets, contexts, labels = skip_gram_with_negative_sampling(
    sequences=sequences,
    context_window=2,
    negative_sample_number=4,
    vocab_size=vocab_size,
)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

  0%|          | 692/511617 [01:27<22:13:04,  6.39it/s]

In [46]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((2,), (2, 5)), (2, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [47]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((2,), (2, 5)), (2, 5)), types: ((tf.int64, tf.int64), tf.int64)>


### Model

In [42]:
num_ns = 4
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                             embedding_dim,
                                             input_length=1,
                                             name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                              embedding_dim,
                                              input_length=num_ns+1)
    
  def call(self, pair):
    target, context = pair
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = tf.einsum('be, bce->bc', word_emb, context_emb)
    return dots

### Training

In [48]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [49]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [50]:
word2vec.fit(dataset, epochs=5, callbacks=[tensorboard_callback])

Epoch 1/5
32/32 [==============================] - 8s 208ms/step - loss: 1.6089 - accuracy: 0.2188
Epoch 2/5
32/32 [==============================] - 6s 202ms/step - loss: 1.5723 - accuracy: 1.0000
Epoch 3/5
32/32 [==============================] - 7s 209ms/step - loss: 1.5280 - accuracy: 1.0000
Epoch 4/5
32/32 [==============================] - 7s 208ms/step - loss: 1.4618 - accuracy: 1.0000
Epoch 5/5
32/32 [==============================] - 6s 200ms/step - loss: 1.3647 - accuracy: 1.0000


In [51]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


## Vectorize code


*   https://sebastianraschka.com/blog/2020/numpy-intro.html#array-math-and-universal-functions
*   https://pythonspeed.com/articles/vectorization-python/

